# Deploy e Serving de Modelos com MLflow


## Objetivo
1. Recuperar o melhor modelo armazenado no MLflow.
2. Realizar deploy local do modelo para servir previsões.
3. Executar inferências online e em batch.
4. Comparar o tempo de resposta e o uso de memória para as inferências.

## Bibliotecas Necessárias
    

In [ ]:

# Manipulação e visualização de dados
import pandas as pd
import numpy as np
import time
import psutil

# MLflow para gerenciamento de modelos e deploy
import mlflow
import mlflow.pyfunc

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")
    

## Recuperando o Melhor Modelo do MLflow

In [ ]:

# Recuperar o ID do experimento e a execução mais recente com o melhor modelo
experiment_name = "Melhor Modelo"  # Nome definido anteriormente
client = mlflow.tracking.MlflowClient()

# Recuperar a última execução do experimento
experiment = client.get_experiment_by_name(experiment_name)
if experiment:
    experiment_id = experiment.experiment_id
    runs = client.search_runs(experiment_id, order_by=["metrics.Acurácia DESC"], max_results=1)
    best_run = runs[0]
    model_uri = f"runs:/{best_run.info.run_id}/melhor_modelo"
    print(f"Melhor modelo encontrado no URI: {model_uri}")
else:
    raise ValueError("Experimento não encontrado no MLflow.")
    

## Realizando o Deploy Local do Modelo

In [ ]:

# Carregar o modelo
modelo = mlflow.pyfunc.load_model(model_uri)

# Testar o modelo carregado com um exemplo
dados_teste = X_test[:5]  # Substitua por amostras reais se necessário
predicoes_teste = modelo.predict(dados_teste)
print(f"Previsões do modelo: {predicoes_teste}")
    

## Inferência em Batch

In [ ]:

# Medir o tempo de inferência em batch
inicio_batch = time.time()
predicoes_batch = modelo.predict(X_test)  # Inferência em todo o conjunto de teste
fim_batch = time.time()

tempo_batch = fim_batch - inicio_batch
print(f"Tempo de inferência em batch: {tempo_batch:.4f} segundos")
    

## Inferência Online

In [ ]:

# Medir o tempo de inferência online
tempos_online = []
for i in range(10):  # Inferência para 10 amostras
    inicio_online = time.time()
    modelo.predict(X_test.iloc[i:i+1])
    fim_online = time.time()
    tempos_online.append(fim_online - inicio_online)

tempo_medio_online = np.mean(tempos_online)
print(f"Tempo médio de inferência online: {tempo_medio_online:.4f} segundos")
    

## Comparação de Desempenho

In [ ]:

# Comparar tempos de inferência
print(f"Tempo de inferência em batch: {tempo_batch:.4f} segundos")
print(f"Tempo médio de inferência online: {tempo_medio_online:.4f} segundos")

# Uso de memória
memoria_inicial = psutil.virtual_memory().used / 1e6  # Memória inicial em MB
modelo.predict(X_test)  # Inferência para medir uso de memória
memoria_final = psutil.virtual_memory().used / 1e6  # Memória final em MB

uso_memoria = memoria_final - memoria_inicial
print(f"Uso de memória para inferência: {uso_memoria:.2f} MB")
    

## Conclusão


Este notebook demonstrou como realizar o deploy de um modelo armazenado no MLflow, executar inferências online e em batch, e comparar o desempenho em termos de tempo e uso de memória.
    